[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/devel/Telegram_Telethon.ipynb)

# Telethon: a comprehensive python 3 library for telegram and MTProto. ![alt text](https://)

The puropose of this notebook is to illustrate the examplesfound on the [documentation for the telethon libraruy](https://telethon.readthedocs.io). as with the rest of the notebooks in this repopsitory, it was developed using [Colaboratory](https://colab.research.google.com) which is a notebook environment that runs off of your google drive, so you don't needto set up a whole notebook envronment and just click away and get gong wth code. 

## Setting up your environment

In [0]:
#@title Notebook parameters { run: "auto", vertical-output: true, display-mode: "form" }
SERVICE_ACCOUNT_FILE = "colab-notebooks-7b143dd37c90.json" #@param {type:"string"}
DELEGATED_ACCOUNT = "hector@lecuanda.com" #@param {type:"string"}
PROJECT = "colab-notebooks" #@param {type:"string"}
DEBUG = True #@param {type:"boolean"}
LOGGING = False #@param {type:"boolean"}
MODULE = 'Telegram-Telethon.ipynb'
VERSION = '0.0.1a0'

if LOGGING:
    import logging
    logging.basicConfig(level=logging.DEBUG)

print('SERVICE_ACCOUNT_FILE: {}'.format(SERVICE_ACCOUNT_FILE))
print('DELEGATED_ACCOUNT: {}'.format(DELEGATED_ACCOUNT))
print('PROJECT: {}'.format(PROJECT))

import os
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT
print('GOOGLE_CLOUD_PROJECT environment set to "{}"'.format(PROJECT))

### Authentication

In [0]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

### Error Reporting

In [0]:
try:
    import googleclouddebugger
[ ]

1
try:
2
    import googleclouddebugger
3
    from google.cloud import error_reporting
4
except ImportError:
5
    !pip -q install google-cloud-error-reporting
6
    !pip -q install google-python-cloud-debugger
7
    from google.cloud import error_reporting
8
    import googleclouddebugger
9
​
10
finally:
11
    erc = error_reporting.Client(PROJECT)    # client = error_reporting.Client??
12
           
13
def simulate_error():
14
    try:
15
        # simulate calling a method that's not defined
16
        raise NameError
17
    except Exception:
18
        erc.report_exception()
19
        
20
if DEBUG:
21
    googleclouddebugger.enable(
22
        module=MODULE,
23
        version=VERSION
24
    )
25
    simulate_error()
26
​
27
 
Héctor Lecuanda
[ ]

1
simulate_error()
Telethon Library
[ ]

1
try:
2
    import telethon
3
except ModuleNotFoundError as e:
4
    !pip -q install telethon
5
    import telethon
6
finally:
7
    from telethon.tl.functions.users import GetFullUserRequest
8
    from telethon.errors import SessionPasswordNeededError
9
    from telethon import TelegramClient
10
    from telethon import utils
11
    if LOGGING:
12
        logging.getLogger('telethon').setLevel(level=logging.WARNING)
13
        prnt('Logging enabled at {}'.format(logging.WARNING))
14
    import getpass
Get your own api_hash and api_id at https;//my.telegram.org these should be unique for your own notebook/application

[ ]

1
#@title App secrets { vertical-output: true, display-mode: "both" }
2
api_hash = "d1d369768ccd7c0888a55d9ef3ad1e86" #@param {type:"string"}
3
api_id = 19887 #@param {type:"integer"}
4
myphone = "+3237414562" #@param {type:"string"}
5
forceSMS = True #@param {type:"boolean"}
6
debug = True #@param {type:"boolean"}
App secrets
api_hash:

d1d369768ccd7c0888a55d9ef3ad1e86
api_id:

19887
myphone:

+3237414562
forceSMS:

debug:

Now wth the connection parameters, app id and hash we can establish a connection to the telegram servers by creating a TelegramClient() please note that

Common error conditions are handled
we should check for the existance of the .session file to avoid constantly hitting the auth API
Some of the parameters to the Telethon.client.start() instance methods, specfically phone_noand auth2fashould be the names of callback functions. The idea is to enable automation by providing a hook to get the required data trough a procedure you define on this function. it could for example invoke the secrets handling api from google cloud encryption management, or query an etcd server in your Kubernetes cluster, or just display a form. in this case the callback will just return the value of the global variable defined on the form above, but you may get creative here.
[ ]

1
def phoneno():
2
    return myphone
3
​
4
def auth2fa():
5
    return getpass.getpass('Enter your 2FA password: ')
6
​
7
def reauthorize():
8
    client.start(phoneno,auth2fa)
9
​
10
​
11
client = TelegramClient('session_name', api_id, api_hash)
12
​
13
if not client.is_connected():
14
    print('Not connected')
15
    if not client.is_user_authorized():
16
        print ('not Authorized')
17
        client.start(phoneno,auth2fa)
18
    else:
19
        print('a')
20
else:
21
    print('b')
22
       
Getting data from the Telegram API
Get your own info
This may be the most basic call.

[ ]

1
me = client.get_me()
2
​
3
me 
4
​
5
if debug:
6
    print(me.user.stringify())
Anolther way of doing it, which wiill get the full user object instead:

[ ]

1
me=client(GetFullUserRequest('me'))
2
​
3
me 
4
​
5
if debug:
6
    print(me.stringify())
7
    debug
Get someone's User object
[ ]
UserID:

+526692120197
Double-click (or enter) to edit

Listing all the dialogs (conversations you have open)
[ ]

1
a=client.get_dialogs()[1]
2
print(a.stringify())
[ ]

1
def categorize_chats(UserD):
2
    ''' retrieves the data about conversation, sorting in 
3
    three different arrays, for users, channels and groups
4
    respectibly. '''
5
    
6
    chatsdata = {}
7
    userchats = []
8
    channels =[]
9
    groups =[]
10
    
11
    for dialog in client.get_dialogs():
12
        if dialog.is_user:  
13
            userchats.append(dialog)
14
        elif dialog.is_channel:
15
            channels.append(dialog)
16
        elif  dialog.is_group:
17
            groups.append(dialog)
18
        else:
19
            print(dialog.stringify)
20
    
21
    chatsdata['userchat'] = userchats
22
    chatsdata['channel'] = channels
23
    chatsdata['group'] = groups
24
​
25
    return chatsdata
26
        
Categorization
[ ]

1
from pprint import pprint
2
​
3
c=categorize_chats('me')
4
d=categorize_chats('me')['userchanel']
5
e=d[1]
6
​
7
e.name
8
​
9
for cat in c:
10
    cat
11
    for i in cat:
12
        print("  {}   {}".format(i,i))
13
    
14
​
Sending a message (you can use 'me' or 'self' to message yourself)
[ ]

1
d.m=client.send_message('mtxto', 'Hello World from Telethon!')
2
m.chat.status.stringify()
3
​
[ ]

1
​
[ ]

1
​
2
​
3
# Sending a file
4
client.send_file('mtxto', '/home/myself/Pictures/holidays.jpg')
5
​
6
# Retrieving messages from a chat
7
from telethon import utils
8
for message in client.iter_messages('mtxto', limit=10):
9
    print(utils.get_display_name(message.sender), message.message)
10
​
11
# Listing all the dialogs (conversations you have open)
12
for dialog in client.get_dialogs(limit=10):
13
    print(utils.get_display_name(dialog.entity), dialog.draft.text)
14
​
15
# Downloading profile photos (default path is the working directory)
16
client.download_profile_photo('mtxto')
17
​
18
# Once you have a message with .media (if message.media)
19
# you can download it using client.download_media():
20
messages = client.get_messages('mtxto')
21
client.download_media(messages[0])

    from google.cloud import error_reporting
except ImportError:
    !pip -q install google-cloud-error-reporting
    !pip -q install google-python-cloud-debugger
    from google.cloud import error_reporting
    import googleclouddebugger

finally:
    erc = error_reporting.Client(PROJECT)    # client = error_reporting.Client??
           
def simulate_error():
    try:
        # simulate calling a method that's not defined
        raise NameError
    except Exception:
        erc.report_exception()
        
if DEBUG:
    googleclouddebugger.enable(
        module=MODULE,
        version=VERSION
    )
    simulate_error()

 

In [0]:
simulate_error()

### Telethon Library

In [0]:
try:
    import telethon
except ModuleNotFoundError as e:
    !pip -q install telethon
    import telethon
finally:
    from telethon.tl.functions.users import GetFullUserRequest
    from telethon.errors import SessionPasswordNeededError
    from telethon import TelegramClient
    from telethon import utils
    if LOGGING:
        logging.getLogger('telethon').setLevel(level=logging.WARNING)
        prnt('Logging enabled at {}'.format(logging.WARNING))
    import getpass

Get your own `api_hash` and  `api_id` at [https;//my.telegram.org](https://my.telegram.org) these should be unique for your own notebook/application  

In [0]:
#@title App secrets { vertical-output: true, display-mode: "both" }
api_hash = "d1d369768ccd7c0888a55d9ef3ad1e86" #@param {type:"string"}
api_id = 19887 #@param {type:"integer"}
myphone = "+3237414562" #@param {type:"string"}
forceSMS = True #@param {type:"boolean"}
debug = True #@param {type:"boolean"}

Now wth the connection parameters, app id and hash we can establish a connection to the telegram servers by creating a `TelegramClient()` please note that


*   Common error conditions are handled
*   we should check for the existance of the `.session` file to avoid constantly hitting the auth API
*   Some of the parameters to the [`Telethon.client.start()`](https://telethon.readthedocs.io/en/latest/telethon.html#module-telethon.telegram_client) instance methods, specfically `phone_no`and `auth2fa`should be the names of callback functions. The idea is to enable automation by providing a hook to get the required data trough a procedure you define on this function. it could for example invoke the secrets handling api from google cloud encryption management, or query an `etcd` server in your Kubernetes cluster, or just display a form. in this case the callback will just return the value of the global variable defined on the form above, but you may get creative here. 



In [0]:
def phoneno():
    return myphone

def auth2fa():
    return getpass.getpass('Enter your 2FA password: ')

def reauthorize():
    client.start(phoneno,auth2fa)


client = TelegramClient('session_name', api_id, api_hash)

if not client.is_connected():
    print('Not connected')
    if not client.is_user_authorized():
        print ('not Authorized')
        client.start(phoneno,auth2fa)
    else:
        print('a')
else:
    print('b')
       

## Getting data from the Telegram API

### Get your own info

This may be the most basic call.

In [0]:
me = client.get_me()

me 

if debug:
    print(me.user.stringify())

Anolther way of doing it, which wiill get the full user object instead:

In [0]:
me=client(GetFullUserRequest('me'))

me 

if debug:
    print(me.stringify())
    debug

### Get someone's `User` object

In [0]:
#title UserID { vertical-output: true, display-mode: "both" }
UserID = "+526692120197" #@param {type:"string"}

g = client(GetFullUserRequest(UserID))

if debug:
    printf(g.stringify())
    if debug:
        print(me.stringify())
        debug 
 

# Listing all the dialogs (conversations you have open)


In [0]:
a=client.get_dialogs()[1]
print(a.stringify())

In [0]:
def categorize_chats(UserD):
    ''' retrieves the data about conversation, sorting in 
    three different arrays, for users, channels and groups
    respectibly. '''
    
    chatsdata = {}
    userchats = []
    channels =[]
    groups =[]
    
    for dialog in client.get_dialogs():
        if dialog.is_user:  
            userchats.append(dialog)
        elif dialog.is_channel:
            channels.append(dialog)
        elif  dialog.is_group:
            groups.append(dialog)
        else:
            print(dialog.stringify)
    
    chatsdata['userchat'] = userchats
    chatsdata['channel'] = channels
    chatsdata['group'] = groups

    return chatsdata
        

### Categorization

In [0]:
from pprint import pprint

c=categorize_chats('me')
d=categorize_chats('me')['userchanel']
e=d[1]

e.name

for cat in c:
    cat
    for i in cat:
        print("  {}   {}".format(i,i))
    



### Sending a message (you can use 'me' or 'self' to message yourself)


In [0]:
d.m=client.send_message('mtxto', 'Hello World from Telethon!')
m.chat.status.stringify()


In [0]:


# Sending a file
client.send_file('mtxto', '/home/myself/Pictures/holidays.jpg')

# Retrieving messages from a chat
from telethon import utils
for message in client.iter_messages('mtxto', limit=10):
    print(utils.get_display_name(message.sender), message.message)

# Listing all the dialogs (conversations you have open)
for dialog in client.get_dialogs(limit=10):
    print(utils.get_display_name(dialog.entity), dialog.draft.text)

# Downloading profile photos (default path is the working directory)
client.download_profile_photo('mtxto')

# Once you have a message with .media (if message.media)
# you can download it using client.download_media():
messages = client.get_messages('mtxto')
client.download_media(messages[0])